<a href="https://colab.research.google.com/github/ssood/ESG/blob/main/Visualising_ESG_scores_from_Sustainalytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualising ESG scores from Sustainalytics
### Adapted from work of Shantala Mukherjee

This  notebook looks at the ESG scores of the stocks in prominent country indices of USA, UK, Japan and Australia. We will obtain the sustainability data from Yahoo! Finance using the Python yfinance library to download historical market data from Yahoo! finance. Yahoo! Finance obtains the ESG scores from Sustainalytics.

In [ ]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
import pandas as pd
import numpy as np
import openpyxl
import altair as alt
import yfinance as yf
from IPython.display import display, HTML

## Read in the index components from the website topforeignstocks.com
### See https://topforeignstocks.com/wp-content/uploads/2021/

##openpyxl is a Python library to read/write Excel 2010 xlsx/xlsm/xltx/xltm files

In [ ]:
#FTSE 100 holdings (United Kingdom)
dfuk = pd.read_excel('https://topforeignstocks.com/wp-content/uploads/2021/01/UK-FTSE-100-Index-Constituents-List-Jan-2021-New.xlsx', engine='openpyxl')

#ASX holdings (Australia)
dfaus = pd.read_excel('https://topforeignstocks.com/wp-content/uploads/2021/01/Australia-ASX-200-Index-Constituents-List-Jan-2021.xlsx', engine='openpyxl')

#Nikkei 225 holdings (Japan)
dfjapan = pd.read_excel('https://topforeignstocks.com/wp-content/uploads/2020/01/Complete-List-of-Constituents-of-Nikkei-225-Jan-2020-Original.xlsx', engine='openpyxl')

#S&P 500  (United States of America)
dfusa = pd.read_excel('https://topforeignstocks.com/wp-content/uploads/2021/01/SP-500-Index-Constituents-List-Jan-1-2021.xlsx', engine='openpyxl')


In [ ]:
dfuk.head() #show the first five rows of the dataframe

,S.No.,Company Name,Ticker
0,1,3I GROUP PLC,III
1,2,ADMIRAL GROUP PLC,ADM
2,3,ANGLO AMERICAN PLC,AAL
3,4,ANTOFAGASTA PLC,ANTO
4,5,ASHTEAD GROUP PLC,AHT


In [ ]:
#To allow Yahoo Finance to find the tickers, we need to modify the ticker codes in the UK, Germany, Japan and Australia dataframes.
#UK tickers need to have '.L', Japan tickers need to have '.T', and Australia tickers need to have '.AX' added at the end.

dfuk['Ticker'] = dfuk['Ticker'].astype(str) + '.L'
dfjapan['Ticker'] = dfjapan['Ticker'].astype(str) + '.T'
dfaus['Ticker'] = dfaus['Ticker'].astype(str) + '.AX'


In [ ]:
dfuk.head()

,S.No.,Company Name,Ticker
0,1,3I GROUP PLC,III.L
1,2,ADMIRAL GROUP PLC,ADM.L
2,3,ANGLO AMERICAN PLC,AAL.L
3,4,ANTOFAGASTA PLC,ANTO.L
4,5,ASHTEAD GROUP PLC,AHT.L


In [ ]:
dfaus.head(10) #show the first ten rows of the dfaus dataframe

,S.No.,Company Name,Ticker
0,1,a2 Milk Company Ltd,A2M.AX
1,2,Adbri Ltd,ABC.AX
2,3,Abacus Property Group,ABP.AX
3,4,AGL Energy Ltd,AGL.AX
4,5,Auckland International Airport Ltd,AIA.AX
5,6,Ampol Ltd,ALD.AX
6,7,Aristocrat Leisure Ltd,ALL.AX
7,8,Als Ltd,ALQ.AX
8,9,Altium Ltd,ALU.AX
9,10,Atlas Arteria,ALX.AX


In [ ]:
dfjapan.head()

,S.No.,Company Name,Ticker,Sector
0,1,ADVANTEST CORP,6857.T,Electric Appliances
1,2,AEON LTD,8267.T,Retail Trade
2,3,AGC INC,5201.T,Glass & Ceramics Products
3,4,AJINOMOTO INC,2802.T,Foods
4,5,ALPS ALPINE LTD,6770.T,Electric Appliances


In [ ]:
dfjapan = dfjapan.drop('Sector',1) # drop the Sector column as it isn't there for the other country stock lists.

## Environment, Social and Governance Scores from Sustainalytics
### The following is taken from Yahoo! Finance. Bold attributed to Shantala Mukherjee
"Sustainalytics’ ESG risk ratings assess the degree to which a company’s enterprise business value is at risk driven by environmental, social and governance issues. The rating employs a two-dimensional framework that combines an assessment of a company’s exposure to industry-specific material ESG issues with an assessment of how well the company is managing those issues. The final ESG risk rating scores are a measure of **unmanaged risk on an absolute scale of 0-100, with a lower score signalling less unmanaged ESG risk**."

## Controversy Level from Sustainalytics
### The following is taken from Yahoo! Finance. Bold attributed to Shantala Mukherjee.
"Sustainalytics’ controversies research identifies companies involved in incidents and events that may negatively affect stakeholders, the environment or the company’s operations. Controversies are rated on a scale from one to five, with **five denoting the most serious controversies with the largest potential impact.**"

In [ ]:
#We will define functions to obtain the environmental, social, governance and total ESG scores of a stock from its ticker

def env(ticker):
    if yf.Ticker(ticker).sustainability is not None:
        return yf.Ticker(ticker).sustainability.loc['environmentScore','Value']
    else:
        return np.NaN

def social(ticker):
    if yf.Ticker(ticker).sustainability is not None:
        return yf.Ticker(ticker).sustainability.loc['socialScore','Value']
    else:
        return np.NaN

def gov(ticker):
    if yf.Ticker(ticker).sustainability is not None:
        return yf.Ticker(ticker).sustainability.loc['governanceScore','Value']
    else:
        return np.NaN

def total(ticker):
    if yf.Ticker(ticker).sustainability is not None:
        return yf.Ticker(ticker).sustainability.loc['totalEsg','Value']
    else:
        return np.NaN

def percentile(ticker):
    if yf.Ticker(ticker).sustainability is not None:
        return yf.Ticker(ticker).sustainability.loc['percentile', 'Value']
    else:
        return np.Nan

def contro(ticker):
    if yf.Ticker(ticker).sustainability is not None:
        return yf.Ticker(ticker).sustainability.loc['highestControversy','Value']
    else:
        return np.NaN

In [ ]:
# We will define a function that combines these scores into a dataframe

def scores(ticker):
    return pd.DataFrame([[ticker,yf.Ticker(ticker).info['longName'],env(ticker),social(ticker),gov(ticker),total(ticker),percentile(ticker),contro(ticker)]],columns=['Ticker','Name','Environmental','Social','Governance','Total ESG','Percentile','Controversy Level'])

## Choose the country to see the list of companies and tickers in the index

In [ ]:
#Country = 'Japan'
Country = 'UK'

In [ ]:
# Choose the appropriate dataframe of companies and tickers according to the country chose above

if Country == 'UK':
    display(HTML(dfuk.to_html()))
elif Country == 'USA':
    display(HTML(dfusa.to_html()))
elif Country == 'Japan':
    display(HTML(dfjapan.to_html()))
else:
    display(HTML(dfaus.to_html()))


## Type in a ticker from any of the lists above
(Please note that the input cell is free text, so you can type in **any** ticker from **any** global index on Yahoo! Finance. The above country stock lists are for ease of reference to look up the exact ticker.)

In [ ]:
#Ticker_code = '6758.T'
#Ticker_code = 'WPP.L'
Ticker_code = 'VOD.L'

### Run the following cell to obtain the ESG scores -- it will take several seconds to run!

In [ ]:
# We will display the dataframe containing the ESG scores

tkr = Ticker_code
df = scores(tkr)
display(HTML(df.to_html()))


,Ticker,Name,Environmental,Social,Governance,Total ESG,Percentile,Controversy Level
0,VOD.L,Vodafone Group Plc,2.72,7.81,6.43,16.95,13.77,3


## Visualise these scores using the Altair statistical visualization library for Python

Run the cell below to see visualisations for these ESG scores. If you do not see the charts, clear the cell output and run the cell again.

In [ ]:
chart1 = alt.Chart(df).mark_bar().encode(
    alt.Y('Ticker:N'),
    alt.X('Total ESG:Q',
        scale=alt.Scale(domain=(0,100))
    ),
    color=alt.Color('Total ESG:Q', scale=alt.Scale(domain = (0,100), scheme='yellowgreenblue')),
    tooltip = [alt.Tooltip('Name:N'),
               alt.Tooltip('Total ESG:Q')
              ]
).interactive()

chart2 = alt.Chart(df).mark_bar().encode(
    alt.X('Percentile:Q',
        scale=alt.Scale(domain=(0,100))
    ),
    alt.Y('Ticker:N'),
    color=alt.Color('Percentile:Q', scale=alt.Scale(domain = (0,100), scheme='yellowgreenblue')),
    tooltip = [alt.Tooltip('Name:N'),
               alt.Tooltip('Percentile:Q')
              ]
).interactive()

chart3 = alt.Chart(df).mark_bar().encode(
    alt.Y('Ticker:N'),
    alt.X('Controversy Level:Q',
        scale=alt.Scale(domain=(0,5))
    ),
    color=alt.Color('Controversy Level:Q', scale=alt.Scale(domain = (0,5), scheme='yellowgreenblue')),
    tooltip = [alt.Tooltip('Name:N'),
               alt.Tooltip('Controversy Level:Q')
              ]
).interactive()

dfesg = df[['Ticker', 'Name','Environmental', 'Social', 'Governance']].copy()

chart4 = alt.Chart(dfesg,title=dfesg['Name'][0]).transform_fold(
    ['Environmental', 'Social', 'Governance'],
    as_=['Attribute', 'Scores']).mark_bar().encode(
    alt.Y('Ticker:N'),
    alt.X('Scores:Q'),
    color='Attribute:N',
    tooltip = [alt.Tooltip('Name:N'),
               alt.Tooltip('Attribute:N'),
               alt.Tooltip('Scores:Q')
              ]
    ).interactive()

alt.vconcat(chart4, chart1, chart2, chart3)

alt.VConcatChart(...)